In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.graphics.gofplots import qqplot
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm_notebook
from itertools import product
from typing import Union

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# Monthly Air Passengers from January 1949 to December 1960
df = pd.read_csv('../data/air-passengers.csv')
df.head()

,Month,Passengers
0,1949-01,112
1,1949-02,118
2,1949-03,132
3,1949-04,129
4,1949-05,121


#### 8.3.2 Forecasting with a SARIMA(p,d,q)(P,D,Q)_m model

In [3]:
# Pg 172

ad_fuller_result = adfuller(df['Passengers'])

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

ADF Statistic: 0.8153688792060441
p-value: 0.9918802434376409


In [4]:
df['Passengers'].tail() #$hape (144,)

139    606
140    508
141    461
142    390
143    432
Name: Passengers, dtype: int64

In [5]:
# Pg 172

df_diff = np.diff(df['Passengers'], n=1)

ad_fuller_result = adfuller(df_diff)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

ADF Statistic: -2.8292668241699968
p-value: 0.05421329028382589


In [6]:
df_diff

array([   6,   14,   -3,   -8,   14,   13,    0,  -12,  -17,  -15,   14,
         -3,   11,   15,   -6,  -10,   24,   21,    0,  -12,  -25,  -19,
         26,    5,    5,   28,  -15,    9,    6,   21,    0,  -15,  -22,
        -16,   20,    5,    9,   13,  -12,    2,   35,   12,   12,  -33,
        -18,  -19,   22,    2,    0,   40,   -1,   -6,   14,   21,    8,
        -35,  -26,  -31,   21,    3,  -16,   47,   -8,    7,   30,   38,
         -9,  -34,  -30,  -26,   26,   13,   -9,   34,    2,    1,   45,
         49,  -17,  -35,  -38,  -37,   41,    6,   -7,   40,   -4,    5,
         56,   39,   -8,  -50,  -49,  -35,   35,    9,  -14,   55,   -8,
          7,   67,   43,    2,  -63,  -57,  -42,   31,    4,  -22,   44,
        -14,   15,   72,   56,   14, -101,  -45,  -49,   27,   23,  -18,
         64,  -10,   24,   52,   76,   11,  -96,  -56,  -45,   43,   12,
        -26,   28,   42,   11,   63,   87,  -16,  -98,  -47,  -71,   42])

In [7]:
# Pg 172

df_diff_diff = np.diff(df_diff, n=1)

ad_fuller_result = adfuller(df_diff_diff)

print(f'ADF Statistic: {ad_fuller_result[0]}')
print(f'p-value: {ad_fuller_result[1]}')

ADF Statistic: -16.38423154246854
p-value: 2.7328918500140445e-29


In [8]:
df_diff_diff

array([   8,  -17,   -5,   22,   -1,  -13,  -12,   -5,    2,   29,  -17,
         14,    4,  -21,   -4,   34,   -3,  -21,  -12,  -13,    6,   45,
        -21,    0,   23,  -43,   24,   -3,   15,  -21,  -15,   -7,    6,
         36,  -15,    4,    4,  -25,   14,   33,  -23,    0,  -45,   15,
         -1,   41,  -20,   -2,   40,  -41,   -5,   20,    7,  -13,  -43,
          9,   -5,   52,  -18,  -19,   63,  -55,   15,   23,    8,  -47,
        -25,    4,    4,   52,  -13,  -22,   43,  -32,   -1,   44,    4,
        -66,  -18,   -3,    1,   78,  -35,  -13,   47,  -44,    9,   51,
        -17,  -47,  -42,    1,   14,   70,  -26,  -23,   69,  -63,   15,
         60,  -24,  -41,  -65,    6,   15,   73,  -27,  -26,   66,  -58,
         29,   57,  -16,  -42, -115,   56,   -4,   76,   -4,  -41,   82,
        -74,   34,   28,   24,  -65, -107,   40,   11,   88,  -31,  -38,
         54,   14,  -31,   52,   24, -103,  -82,   51,  -24,  113])

In [9]:
def optimize_SARIMA(endog: Union[pd.Series, list], order_list: list, d: int, D: int, s: int) -> pd.DataFrame:
    
    results = []
    
    for order in tqdm_notebook(order_list):
        try: 
            model = SARIMAX(
                endog, 
                order=(order[0], d, order[1]),
                seasonal_order=(order[2], D, order[3], s),
                simple_differencing=False).fit(disp=False)
        except:
            continue
            
        aic = model.aic
        results.append([order, aic])
        
    result_df = pd.DataFrame(results)
    result_df.columns = ['(p,q,P,Q)', 'AIC']
    
    #Sort in ascending order, lower AIC is better
    result_df = result_df.sort_values(by='AIC', ascending=True).reset_index(drop=True)
    
    return result_df

In [ ]:
# Pg 174

ps = range(0, 13, 1)
qs = range(0, 13, 1)
Ps = range(0, 4, 1)
Qs = range(0, 4, 1)

SARIMA_order_list = list(product(ps, qs, Ps, Qs))

train = df['Passengers'][:-12] # len 132
#test = df['Passengers'][-12:]  # len 12

d = 2
D = 0
s = 12

SARIMA_result_df = optimize_SARIMA(train, SARIMA_order_list, d, D, s)
SARIMA_result_df

  0%|          | 0/2704 [00:00<?, ?it/s]

In [ ]:
SARIMA_result_df[10:20]

***************************************************************************
Pick a Model and analyze residuals

We pick SARIMA(1,2,0)(3,0,2)_12, which has the lowest AIC

In [ ]:
SARIMA_model = SARIMAX(train, order=(1,2,0), seasonal_order=(3,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'

print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid

lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))

lb_dataframe

SARIMA(1,2,0)(3,0,2)_12, Ljung-Box test we can reject null hypothesis for lags 3 to lag 20  
We pick now SARIMA(3,2,1)(2,0,1)_12

In [ ]:
SARIMA_model = SARIMAX(train, order=(3,2,1), seasonal_order=(2,0,1,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'

print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid

lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))

lb_dataframe

SARIMA(3,2,1)(2,0,1)_12, Ljung-Box test we can reject null hypothesis for lags 4 to lag 20  
We pick now SARIMA(2,2,1)(2,0,2)_12

In [ ]:
SARIMA_model = SARIMAX(train, order=(2,2,1), seasonal_order=(2,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'

print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid

lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))

lb_dataframe

SARIMA(2,2,1)(2,0,2)_12, Ljung-Box test we can reject null hypothesis for lags 1 to lag 20  
We pick now SARIMA(1,2,3)(2,0,2)_12	

In [ ]:
SARIMA_model = SARIMAX(train, order=(1,2,3), seasonal_order=(2,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'

print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid

lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))

lb_dataframe

SARIMA(1,2,3)(2,0,2)_12	, Ljung-Box test we can reject null hypothesis for lags 1 to lag 20  
We pick now SARIMA(3,2,1)(3,0,1)_12	


In [ ]:
SARIMA_model = SARIMAX(train, order=(3,2,1), seasonal_order=(3,0,1,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'

print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid
lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))
lb_dataframe

SARIMA(3,2,1)(3,0,1)_12	, Ljung-Box test we can reject null hypothesis for lags 4 to lag 20  
We pick now SARIMA(3,2,1)(3,0,2)_12	

In [ ]:
SARIMA_model = SARIMAX(train, order=(3,2,1), seasonal_order=(3,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'
print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid
lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))
lb_dataframe

SARIMA(3,2,1)(3,0,2)_12	  , Ljung-Box test we can reject null hypothesis for lag and for lags 4 to lag 20   
We pick now SARIMA(1,2,2)(2,0,2)_12 

In [ ]:
SARIMA_model = SARIMAX(train, order=(1,2,2), seasonal_order=(2,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'
print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid
lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))
lb_dataframe

SARIMA(1,2,2)(2,0,2)_12 , Ljung-Box test we can reject null hypothesis for lag and for lags 1 to lag 20   
We pick now SARIMA(2,2,0)(2,0,2)_12 

In [ ]:
SARIMA_model = SARIMAX(train, order=(2,2,0), seasonal_order=(2,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'
print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid
lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))
lb_dataframe

SARIMA(2,2,0)(2,0,2)_12  , Ljung-Box test we can reject null hypothesis for lag and for lags 1 to lag 20   
We pick now SARIMA(2,2,3)(1,0,0)_12 

In [ ]:
SARIMA_model = SARIMAX(train, order=(2,2,3), seasonal_order=(1,0,0,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'
print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))


In [ ]:
residuals = SARIMA_model_fit.resid
lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))
lb_dataframe

SARIMA(2,2,3)(1,0,0)_12 , Ljung-Box test we can reject null hypothesis for lag and for lags 2 to lag 20   
We pick now SARIMA(2,2,3)(3,0,2)_12 

In [ ]:
SARIMA_model = SARIMAX(train, order=(2,2,3), seasonal_order=(3,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'
print(SARIMA_model_fit.summary())

In [ ]:
SARIMA_model_fit.plot_diagnostics(figsize=(10,8))

In [ ]:
residuals = SARIMA_model_fit.resid
lb_dataframe = acorr_ljungbox(residuals, np.arange(1, 11, 1))
lb_dataframe

SARIMA(2,2,3)(3,0,2)_12  , Ljung-Box test we can reject null hypothesis for lag and for lags 2 to lag 20   

We tried 10 models and none capture all the information from the data. Still, let's pick the first one.

*******************************************************************************************
Forecast with model

In [ ]:
SARIMA_model = SARIMAX(train, order=(1,2,0), seasonal_order=(3,0,2,12), simple_differencing=False)
SARIMA_model_fit = SARIMA_model.fit(disp=False, method='nm') # method='powell'
#print(SARIMA_model_fit.summary())

In [ ]:
test = df.iloc[-12:]

test['naive_seasonal'] = df['Passengers'].iloc[120:132].values
test

In [ ]:
# Pg 175

SARIMA_pred = SARIMA_model_fit.get_prediction(132, 143).predicted_mean

test['SARIMA_pred'] = SARIMA_pred
test

In [ ]:
# Figure 8.13 in pg 176

fig, ax = plt.subplots()

ax.plot(df['Month'], df['Passengers'])
ax.plot(test['Passengers'], 'b-', label='actual')
ax.plot(test['naive_seasonal'], 'r:', label='naive seasonal')
#ax.plot(test['ARIMA_pred'], 'k--', label='ARIMA(11,2,3)')
ax.plot(test['SARIMA_pred'], 'g-.', label='SARIMA(1,2,0)(3,0,2,12)')

ax.set_xlabel('Date')
ax.set_ylabel('Number of air passengers')
ax.axvspan(132, 143, color='#808080', alpha=0.2)

ax.legend(loc=2)

plt.xticks(np.arange(0, 145, 12), np.arange(1949, 1962, 1))
ax.set_xlim(120, 143)

fig.autofmt_xdate()
plt.tight_layout()


In [ ]:
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
mape_naive_seasonal = mape(test['Passengers'], test['naive_seasonal'])
#mape_ARIMA = mape(test['Passengers'], test['ARIMA_pred'])
mape_SARIMA = mape(test['Passengers'], test['SARIMA_pred'])

print(mape_naive_seasonal, mape_SARIMA)

In [ ]:
# Figure 8.14 in pg 177

fig, ax = plt.subplots()

x = ['naive seasonal', 'ARIMA(11,2,3)', 'SARIMA(2,1,1)(1,1,2,12)']
y = [mape_naive_seasonal, mape_ARIMA, mape_SARIMA]

ax.bar(x, y, width=0.4)
ax.set_xlabel('Models')
ax.set_ylabel('MAPE (%)')
ax.set_ylim(0, 15)

for index, value in enumerate(y):
    plt.text(x=index, y=value + 1, s=str(round(value,2)), ha='center')

plt.tight_layout()

plt.savefig('figures/CH08_F14_peixeiro.png', dpi=300)

SARIMA 2.85% off vs ARIMA 3.84% off